# Flower Identifier

In [7]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tqdm as notebook_tqdm

In [8]:
dataset, info = tfds.load('oxford_flowers102', with_info=True, as_supervised=True)

In [9]:
train_dataset, test_dataset, val_dataset = dataset['train'], dataset['test'], dataset['validation']

In [10]:
IMG_SIZE = 224

def preprocess_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))  # Resize the image to 224x224
    image = tf.cast(image, tf.float32) / 255.0  # Normalize the pixel values to [0, 1]
    return image, label

In [11]:
train_dataset = train_dataset.map(preprocess_image).batch(32)
test_dataset = test_dataset.map(preprocess_image).batch(32)
val_dataset = val_dataset.map(preprocess_image).batch(32)

In [12]:
# Load model
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
# Freeze the base model
base_model.trainable = False

# Build the final model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(102, activation='softmax')
])

In [14]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])